<a href="https://colab.research.google.com/github/julianovale/project_trains/blob/master/AlgoritmoCarla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#
# Matrix
#

class Matrix:
    def __init__(self, order, beg, ind, val):
        self.order = order if type(order) is tuple else (order, order)
        self.beg   = beg
        self.ind   = ind
        self.val   = val

    @staticmethod
    def identity(order):
        return Matrix((order, order), list(range(0, order + 1)), list(range(0, order)), [1] * order)

In [20]:
import threading
import queue

#from matrix import Matrix

RID = 10000000
PID =  1000000
VID =  2000000

#
# LazyExpressionLeafNode
#

class LazyExpressionLeafNode:
    def __init__(self, matrix):
        self.matrix = matrix
        self.size   = matrix.order[0]

    def evaluate(self, row):
        start = self.matrix.beg[row]
        end   = self.matrix.beg[row + 1]

        return self.matrix.ind[start:end], self.matrix.val[start:end]

#
# LazyExpressionInnerNode
#

class LazyExpressionInnerNode:
    def __init__(self, operand, left, right):
        self.operand = operand
        self.left    = left
        self.right   = right
        self.size    = left.size if operand == '+' else left.size * right.size
        self.matrix  = {}

    def evaluate(self, row):
        # verifica se a linha do nó já foi calculada
        if row in self.matrix:
            return self.matrix[row]

        # determina as linhas dos nós sucessores
        lrow = row
        rrow = row
        rsz  = self.right.size

        if self.operand != '+':
            lrow //= rsz
            rrow  %= rsz

        # calcula as linhas dos nós sucessores
        lind, lval = self.left.evaluate(lrow)
        rind, rval = self.right.evaluate(rrow)

        # calcula a linha do nó e armazena o resultado
        if self.operand == '+':
            return self.matrix.setdefault(row, (lind + rind, lval + rval))

        nnz = len(lind)
        ind = []
        val = []

        if self.operand == '⊗':
            for i in range(nnz):
                ind.extend([lind[i] * rsz + ind for ind in rind])
                val.extend([lval[i] * val for val in rval])
        else:
            # self.operand == '⊘'
            for i in range(nnz):
                q = lind[i] * rsz
                v = lval[i]
                for k in range(len(rind)):
                    if rval[k] == v % RID:
                        ind.append(q + rind[k])
                        val.append(v)

        return self.matrix.setdefault(row, (ind, val))
    
    def getnnz(self):
        nnz = 0

        for i in self.matrix:
            nnz += len(self.matrix[i][0])
        
        return nnz

#
# LazyExpressionTree
#

ENDWORKER = 'ENDWORKER'

class LazyExpressionTree:
    def __init__(self):
        self.root = None
        self.__q  = queue.Queue()
        self.__v  = {}

    def build(self, routes, j, i):
        left  = self.__build_R(routes, j) if j > 1 else LazyExpressionLeafNode(self.__R(routes, 0))
        right = self.__build_T(i) if i > 1 else LazyExpressionLeafNode(self.__T(0))

        self.root = LazyExpressionInnerNode('⊘', left, right)

    def __build_R(self, routes, k):
        # R1 + R2
        R1 = self.__R(routes, 0)
        m1 = R1.order[0]
        I1 = Matrix.identity(m1)

        R2 = self.__R(routes, 1)
        m2 = R2.order[0]
        I2 = Matrix.identity(m2)

        m1 *= m2

        left  = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(R1), LazyExpressionLeafNode(I2))
        right = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(I1), LazyExpressionLeafNode(R2))
        node  = LazyExpressionInnerNode('+', left, right)

        # + R3 + R4 ... Rk
        for j in range(2, k):
            Rj = self.__R(routes, j)
            m2 = Rj.order[0]
            Ij = Matrix.identity(m2)

            left  = LazyExpressionInnerNode('⊗', node, LazyExpressionLeafNode(Ij))
            right = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(Matrix.identity(m1)), LazyExpressionLeafNode(Rj))
            node  = LazyExpressionInnerNode('+', left, right)

            m1 *= m2

        return node

    def __build_T(self, k):
        m = 2
        I = Matrix.identity(m)

        # T1 + T2
        left  = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(self.__T(0)), LazyExpressionLeafNode(I))
        right = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(I), LazyExpressionLeafNode(self.__T(1)))
        node  = LazyExpressionInnerNode('+', left, right)

        # + T3 + T4 ... Tk
        for i in range(2, k):
            m *= 2
            Ii = Matrix.identity(m)

            left  = LazyExpressionInnerNode('⊗', node, LazyExpressionLeafNode(I))
            right = LazyExpressionInnerNode('⊗', LazyExpressionLeafNode(Ii), LazyExpressionLeafNode(self.__T(i)))
            node  = LazyExpressionInnerNode('+', left, right)

        return node

    def __R(self, routes, j):
        val = routes[j]

        rid = RID * (j + 1)
        val = [rid + (PID if v > 0 else VID) + abs(v) for v in val]

        n = len(val)
        m = n + 1

        return Matrix(order = (m, m), beg = list(range(0, m)) + [n], ind = list(range(1, m)), val = val)

    def __T(self, i):
        i += 1
        return Matrix(order = (2, 2), beg = [0, 1, 2], ind = [1, 0], val = [PID + i, VID + i])

    def worker(self, i):
        cnt = 0
        while True:
            row = self.__q.get()

            if row == ENDWORKER:
                self.__q.task_done()
                break

            # gera o identificador único
            cnt += 1
            uid  = (i, cnt)

            # verifica se a linha ainda não foi calculada
            if self.__v.setdefault(row, uid) == uid:
                # calcula a linha
                ind = self.root.evaluate(row)[0]

                # coloca na fila as linhas alcançáveis
                for row in ind:
                    self.__q.put(row)

            self.__q.task_done()

    def evaluate(self, w = None):
        if w is None:
            # w = multiprocessing.cpu_count()
            w = 3

        # cria os workers para a avaliação da expressão
        workers = []

        for i in range(w):
            t = threading.Thread(target = self.worker, args = (i,))
            workers.append(t)
            t.start()

        # inicia a avaliação da expressão
        self.__q.put(0)
        self.__q.join()

        # finaliza os workers
        for t in workers:
            self.__q.put(ENDWORKER)

        self.__q.join()

    def __str__(self):
        def todatastr(val):
            out  = 'L' + str(val // RID) + '.'
            val %= RID
            out += ('p' if val < VID else 'v') + str(val % PID)
            return out

        A = self.root.matrix
        return '\n'.join([str(r) + '\t' + '\t'.join([str(t[0]) + ':' + todatastr(t[1]) for t in zip(A[r][0], A[r][1])]) for r in sorted(A.keys())])

In [21]:
import sys
import time

# from railway import LazyExpressionTree

start = time.process_time()

r = LazyExpressionTree()
#r.build([[2, -1, 3, -2, -3], [1, -2, -1]], 2, 3)
r.build([[1, 3, -1, 4, -3, -4],[2, 3, -2, 5, -3, -5],[5, 3, -5, 1, -3, -1]], 3, 5)
#r.build([[1, 3, -1, 4, -3, -4],[2, 3, -2, 5, -3, -5],[5, 3, -5, 1, -3, -1]], 3, 5)
#r.build([[1, -1],[1, -1]], 2, 1)
#r.build([[3, -2, -12, 7, 8, -3, -7, 10, -8, 11, -10, -11], [2, -1, 9, -2, 7, 8, -9, -7, 10, -8, 12, 11, -10, -11], [5, -10, 4, -5, -13, 2, -4, 1, -2, -1], [10, -11, 5, -10, 6, -5, 7, 9, -6, -7, 2, -9, 13, 1, -2, -1]], 4, 13)
#r.build([[1, 2, -1, 5, -2, 6, -5, 7, -6, -7], [1, 2, -1, 3, -2, 7, -3, -7], [7, 4, -7, 2, -4, 1, -2, -1]], 3, 7)
#r.build([[3, -2, 7, 8, -3, -7, 10, -8, 11, -10, -11],[2, -1, 9, -2, 7, 8, -9, -7, 10, -8, 11, -10, -11],[5, -10, 4, -5, 2, -4, 1, -2, -1],[10, -11, 5, -10, 6, -5, 7, 9, -6, -7, 2, -9, 1, -2, -1]], 4, 11)
#r.build([[1, 2, -1, 3, -2, 4, -3, -4],[5, 6, -5, 7, -6, 8, -7, -8]], 2, 8)1
#r.build([[3, -2, -12, 7, 8, -3, -7, 10, -8, 11, -10, -11], [2, -1, 9, -2, 7, 8, -9, -7, 10, -8, 12, 11, -10, -11],[5, -10, 4, 5, -13, 2, -4, 1, -2, -1],[10, -11, 5, -10, 6, -5, 7, 9, -6, -7, 2, -9, 13, 1, -2, -1]], 4, 9)

#r.build([[1, 2, -1, 3, -2, -3], [2, 1, -2, -1]], 2, 3)
#r.build([[1, 5, -1, 7, -5, -7],[2, 4, -2, 5, -4, 6, -5, 9, -6, -9],[3, 4, -3, 5, -4, 6, -5, 8, -6, -8],[8, 6, -8, 5, -6, 1, -5, -1]], 4, 9)
#r.build([[2, 3, -2, -12, 7, 8, -3, -7, 10, -8, 11, -10, -11], [1, 2, -1, 9, -2, 7, 8, -9, -7, 10, -8, 12, 11, -10, -11], [10, 5, -10, 4, -5, -13, 2, -4, 1, -2, -1], [11, 10, -11, 5, -10, 6, -5, 7, 9, -6, -7, 2, -9, 13, 1, -2, -1]], 4, 13)

r.evaluate()

print(time.process_time() - start)

print(r.root.size)
print(r.root.getnnz())
#print(r)

with open('result.txt', 'w') as f:
    _stdout = sys.stdout
    sys.stdout = f
    print(r)
    sys.stdout = _stdout

0.005993966000000128
10976
300


In [22]:
from google.colab import files
files.download('result.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>